# Módulo 4 - Big Data

## Tarea 3

Autor: Luis Felipe Cubero Vargas

In [1]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "./Tarea3/postgresql-42.2.8.jar") \
    .config("spark.executor.extraClassPath", "./Tarea3/postgresql-42.2.8.jar") \
    .getOrCreate()

# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/") \
    .option("user", "postgres") \
    .option("password", "root") \
    .option("dbtable", "bigdata.transactions") \
    .load()

df.show()

+---+-----------+------+-------------------+
| id|customer_id|amount|       purchased_at|
+---+-----------+------+-------------------+
|  1|          1|    55|2017-03-01 09:00:00|
|  2|          1|   125|2017-03-01 10:00:00|
|  3|          1|    32|2017-03-02 13:00:00|
|  4|          1|    64|2017-03-02 15:00:00|
|  5|          1|   128|2017-03-03 10:00:00|
|  6|          2|   333|2017-03-01 09:00:00|
|  7|          2|   334|2017-03-01 09:01:00|
|  8|          2|   333|2017-03-01 09:02:00|
|  9|          2|    11|2017-03-03 20:00:00|
| 10|          2|    44|2017-03-03 20:15:00|
+---+-----------+------+-------------------+



Leer datos directamente rara vez es el objetivo primario. Las dos tareas típicas son: transformar algunas de las columnas en representaciones modificadas de las mismas, o bien, agregar grupos de filas en una. A manera de ejemplo, podemos asumir que se quiere la información del total de compras hechas por clientes.

Spark provee ciertas transformaciones básicas. Por ejemplo, transformar fechas a una representación con formato específico, para poder truncar la una estampilla de tiempo a nivel de día. En este caso, se utiliza `date_format` en el módulo `pyspark.sql.functions`. El siguiente código crea una columna nueva después de aplicar la transformación.

In [10]:
formatted_df = df.withColumn("date_string", date_format(col("purchased_at"), 'MM/dd/yyyy'))
formatted_df.show()

+---+-----------+------+-------------------+-----------+
| id|customer_id|amount|       purchased_at|date_string|
+---+-----------+------+-------------------+-----------+
|  1|          1|    55|2017-03-01 09:00:00| 03/01/2017|
|  2|          1|   125|2017-03-01 10:00:00| 03/01/2017|
|  3|          1|    32|2017-03-02 13:00:00| 03/02/2017|
|  4|          1|    64|2017-03-02 15:00:00| 03/02/2017|
|  5|          1|   128|2017-03-03 10:00:00| 03/03/2017|
|  6|          2|   333|2017-03-01 09:00:00| 03/01/2017|
|  7|          2|   334|2017-03-01 09:01:00| 03/01/2017|
|  8|          2|   333|2017-03-01 09:02:00| 03/01/2017|
|  9|          2|    11|2017-03-03 20:00:00| 03/03/2017|
| 10|          2|    44|2017-03-03 20:15:00| 03/03/2017|
+---+-----------+------+-------------------+-----------+



En el caso que necesitemos crear una función que no es parte de la biblioteca estándar en Spark, es posible definir funciones creadas por el usuario (User Defined Functions o *udf*). La noción básica de una `udf` en Spark un lambda acompañado por el tipo de dato retornado. Lo anterior es estrictamente necesario en lenguajes con un sistema de tipos débil.

Además, es una manera cómoda de encapsular funciones de Python que deben ser aplicadas a celdas del *Dataframe*, pero aún no tienen su implementación en las bibliotecas.

El siguiente ejemplo muestra la columna creada previamente (tipo `string`) y la transforma en un tipo `DateType` propio de Spark.

In [11]:
string_to_date = \
    udf(lambda text_date: datetime.strptime(text_date, '%m/%d/%Y'),
        DateType())

typed_df = formatted_df.withColumn("date", string_to_date(formatted_df.date_string))
typed_df.show()
typed_df.printSchema()


+---+-----------+------+-------------------+-----------+----------+
| id|customer_id|amount|       purchased_at|date_string|      date|
+---+-----------+------+-------------------+-----------+----------+
|  1|          1|    55|2017-03-01 09:00:00| 03/01/2017|2017-03-01|
|  2|          1|   125|2017-03-01 10:00:00| 03/01/2017|2017-03-01|
|  3|          1|    32|2017-03-02 13:00:00| 03/02/2017|2017-03-02|
|  4|          1|    64|2017-03-02 15:00:00| 03/02/2017|2017-03-02|
|  5|          1|   128|2017-03-03 10:00:00| 03/03/2017|2017-03-03|
|  6|          2|   333|2017-03-01 09:00:00| 03/01/2017|2017-03-01|
|  7|          2|   334|2017-03-01 09:01:00| 03/01/2017|2017-03-01|
|  8|          2|   333|2017-03-01 09:02:00| 03/01/2017|2017-03-01|
|  9|          2|    11|2017-03-03 20:00:00| 03/03/2017|2017-03-03|
| 10|          2|    44|2017-03-03 20:15:00| 03/03/2017|2017-03-03|
+---+-----------+------+-------------------+-----------+----------+

root
 |-- id: integer (nullable = true)
 |-- cu

Para sumar los datos podemos utilizar el concepto común de agrupamiento en SQL. Spark posee una agrupación de `groupBy`, directamente. En este caso, queremos sumar todas las compras por cliente y día:

In [13]:
sum_df = typed_df.groupBy("customer_id", "date").sum()
sum_df.show()

+-----------+----------+-------+----------------+-----------+
|customer_id|      date|sum(id)|sum(customer_id)|sum(amount)|
+-----------+----------+-------+----------------+-----------+
|          2|2017-03-01|     21|               6|       1000|
|          1|2017-03-02|      7|               2|         96|
|          1|2017-03-03|      5|               1|        128|
|          1|2017-03-01|      3|               2|        180|
|          2|2017-03-03|     19|               4|         55|
+-----------+----------+-------+----------------+-----------+



Spark sumará todas las columnas **que no se encuentren** especificadas en la operación `groupBy`. Algunos resultados no tienen interpretación últil. Por ejemplo, sumar la columna `customer_id` no da ningún valor agregado. Finalmente, es posible dar a las columnas un nombre más amigable con los consumidores de los datos.

In [14]:
stats_df = \
    sum_df.select(
        col('customer_id'),
        col('date'),
        col('sum(amount)').alias('amount'))

stats_df.printSchema()
stats_df.show()


root
 |-- customer_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- amount: long (nullable = true)

+-----------+----------+------+
|customer_id|      date|amount|
+-----------+----------+------+
|          2|2017-03-01|  1000|
|          1|2017-03-02|    96|
|          1|2017-03-03|   128|
|          1|2017-03-01|   180|
|          2|2017-03-03|    55|
+-----------+----------+------+



Spark permite cargar información de múltiples fuentes. A continuación se muestra como cargar datos de un archivo CSV que contiene los nombres de los clientes, así como la moneda en que realizan transacciones. Nótese que el CSV no tiene información de tipos de datos, por lo que es buena práctica agregarlos explícitamente.

In [15]:
from pyspark.sql.types import IntegerType, StringType, StructField, StructType

names_df = spark \
    .read \
    .format("csv") \
    .option("path", "names.csv") \
    .option("header", True) \
    .schema(StructType([
                StructField("id", IntegerType()),
                StructField("name", StringType()),
                StructField("currency", StringType())])) \
    .load()

names_df.printSchema()
names_df.show()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- currency: string (nullable = true)

+---+----+--------+
| id|name|currency|
+---+----+--------+
|  1|John|     CRC|
|  2|Jane|     EUR|
+---+----+--------+



Una vez que la información fue cargada, la fuente particular no es relevante. A continuación se muestra como podemos enriquerecer la información utilizando la funcion **JOIN** entre *data frames*.

In [16]:
joint_df = stats_df.join(names_df, stats_df.customer_id == names_df.id)
joint_df.printSchema()
joint_df.show()

root
 |-- customer_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- amount: long (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- currency: string (nullable = true)

+-----------+----------+------+---+----+--------+
|customer_id|      date|amount| id|name|currency|
+-----------+----------+------+---+----+--------+
|          2|2017-03-01|  1000|  2|Jane|     EUR|
|          1|2017-03-02|    96|  1|John|     CRC|
|          1|2017-03-03|   128|  1|John|     CRC|
|          1|2017-03-01|   180|  1|John|     CRC|
|          2|2017-03-03|    55|  2|Jane|     EUR|
+-----------+----------+------+---+----+--------+



# Ejercicio
- Cree un pequeño generador de transacciones. Para ello, puede utilizar funciones de numpy o pandas, para crear las transacciones nuevas en memoria y, posteriormente, puede cargarlas a un *Spark Dataframe* que después debe insertar en la base de datos.
- Con los datos nuevos, ejecute el código de éste notebook.

In [9]:
df.show()

+---+-----------+------+-------------------+
| id|customer_id|amount|       purchased_at|
+---+-----------+------+-------------------+
|  1|          1|    55|2017-03-01 09:00:00|
|  2|          1|   125|2017-03-01 10:00:00|
|  3|          1|    32|2017-03-02 13:00:00|
|  4|          1|    64|2017-03-02 15:00:00|
|  5|          1|   128|2017-03-03 10:00:00|
|  6|          2|   333|2017-03-01 09:00:00|
|  7|          2|   334|2017-03-01 09:01:00|
|  8|          2|   333|2017-03-01 09:02:00|
|  9|          2|    11|2017-03-03 20:00:00|
| 10|          2|    44|2017-03-03 20:15:00|
| 11|          1|    50|2017-03-04 21:15:00|
+---+-----------+------+-------------------+



In [8]:
df.columns

['id', 'customer_id', 'amount', 'purchased_at']

In [7]:
newRow = spark.createDataFrame([(11,1,50,'2017-03-04 21:15:00')], df.columns)
df = df.union(newRow)
df.show()

+---+-----------+------+-------------------+
| id|customer_id|amount|       purchased_at|
+---+-----------+------+-------------------+
|  1|          1|    55|2017-03-01 09:00:00|
|  2|          1|   125|2017-03-01 10:00:00|
|  3|          1|    32|2017-03-02 13:00:00|
|  4|          1|    64|2017-03-02 15:00:00|
|  5|          1|   128|2017-03-03 10:00:00|
|  6|          2|   333|2017-03-01 09:00:00|
|  7|          2|   334|2017-03-01 09:01:00|
|  8|          2|   333|2017-03-01 09:02:00|
|  9|          2|    11|2017-03-03 20:00:00|
| 10|          2|    44|2017-03-03 20:15:00|
| 11|          1|    50|2017-03-04 21:15:00|
+---+-----------+------+-------------------+

